In [1]:
import networkx as nx
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output


In [7]:
G = nx.read_graphml("cajamarcaSimpleRoads.graphml")

In [3]:
m = folium.Map(location=[-7.165, -78.5], zoom_start=10)
display(m)

In [11]:
# %%
import math
import networkx as nx
import folium
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Heurística de Haversine para A*
def haversine_heuristic(u, v):
    lat1, lon1 = G.nodes[u]['lat'], G.nodes[u]['lon']
    lat2, lon2 = G.nodes[v]['lat'], G.nodes[v]['lon']
    R = 6371.0
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlambda/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))

# Controles
node_list = list(G.nodes)
orig = widgets.Dropdown(options=node_list, description='Origen:')
dest = widgets.Dropdown(options=node_list, description='Destino:')
algo = widgets.ToggleButtons(options=[('Dijkstra','dij'),('A*','astar')],
                             description='Algoritmo:')
btn = widgets.Button(description='Calcular Ruta', button_style='success')
out = widgets.Output()

def plot_route(_):
    # 1) Limpiar salida anterior
    clear_output(wait=True)
    # 2) Volver a mostrar controles
    display(widgets.VBox([orig, dest, algo, btn, out]))
    out.clear_output()

    # 3) Calcular ruta según algoritmo
    if algo.value == 'dij':
        path = nx.shortest_path(G, orig.value, dest.value, weight='dist')
    else:
        path = nx.astar_path(G, orig.value, dest.value,
                             heuristic=haversine_heuristic,
                             weight='dist')

    coords = [(G.nodes[n]['lat'], G.nodes[n]['lon']) for n in path]

    # 4) Crear mapa nuevo
    m = folium.Map(location=coords[0], zoom_start=12)
    folium.Marker(coords[0], tooltip="Origen", icon=folium.Icon(color='blue')).add_to(m)
    folium.Marker(coords[-1], tooltip="Destino", icon=folium.Icon(color='red')).add_to(m)
    folium.PolyLine(coords, color='green', weight=5, opacity=0.8).add_to(m)

    # 5) Mostrar mapa incrustado
    map_html = m._repr_html_()
    with out:
        display(HTML(map_html))

# 6) Conectar evento y mostrar controles por primera vez
btn.on_click(plot_route)
display(widgets.VBox([orig, dest, algo, btn, out]))
